In [1]:
import os

os.environ["CONFIG_APP_DIR"] = "tests/test_custom/config_app/"

In [2]:
import sys
import os
import pickle
os.chdir("..")
from omegaconf import OmegaConf

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine


In [3]:
from etl_pipeline.config import columns_namespace as cn

In [4]:
# alert = load_alert()

In [5]:
# payload = load_alert()


# Definition

In [13]:
from pipelines.ms.wm_address_pipeline import MSPipeline


In [39]:
with open(f'notebooks/sample/wm_address_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)
payload_json = payload_json['alertPayload']


payload_json['match_ids'] = [i for i in range(len(payload_json[cn.ALERT_FIELD][cn.MATCH_RECORDS]))]


In [40]:
payload = payload_json

In [41]:
engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

## transform standardized to cleansed

In [42]:
payload = pipeline.transform_standardized_to_cleansed(payload)

# Transform standardized to application

In [43]:
new_payloads = pipeline.transform_cleansed_to_application(payload)

### payload

In [44]:
for payload in new_payloads:
    print('p')
    for match in payload['alert']['matchRecords']:
        print('match')
        print("++++++++")
        print(match["ap_all_dobs_aggregated"], match["wl_all_dobs_aggregated"])
        try:
            print(match["ap_all_residencies_aggregated"], match["wl_all_residencies_aggregated"])
        except:
            break


p
match
++++++++
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA'] ['US']
p
match
++++++++
['02/31/1900', '04/31/1910'] ['MAY 6, 1981']
['UNITED STATES OF AMERICA'] ['PL']


In [45]:
import pickle
with open("tests/shared/parsed_payload.pkl", "wb") as f:
    pickle.dump(new_payloads, f)

In [ ]:
new_dict = {}
def return_key(dict_, prefix):
    
    if isinstance(dict_, list):
        for num, i in enumerate(dict_):
            return_key(i, prefix +f"[{num}]")
        return
    if isinstance(dict_, str) or dict_ is None:
        new_dict[prefix] = dict_
        return

    for key in dict_:
        if prefix:
            basic_prefix = prefix + "." + key
        else:
            basic_prefix = key
        return_key(dict_[key], basic_prefix)

In [ ]:
return_key(payload, "")

In [ ]:
with open(f'notebooks/sample/alert_in_payload_format.json', 'w') as file:
    json.dump(new_dict ,file)

In [ ]:
payload_json = PayloadLoader().load_payload_from_json(new_dict)

In [ ]:
payload_json == payload

In [ ]:
with open(f'notebooks/sample/alert.json', 'r') as file:
    payload = json.loads(file.read())

In [ ]:
payload